In [ ]:
from google.colab import files
uploaded = files.upload()
import polars as pl

strikes = pl.read_csv("faa_strikes.txt", separator="\t")
strikes.head()

In [ ]:
#question1
running_total_strikes = (
    strikes.with_columns(
        # Parse datetime, then keep only the date part
        pl.col("Collision Date and Time")
          .str.to_datetime(strict=False)
          .dt.date()
          .alias("date")
    )
    .groupby("date", maintain_order=True)
    .agg(pl.col("Number of Strikes").fill_null(0).sum().alias("daily_strikes"))
    .sort("date")
    .with_columns(
        # Window-style cumulative sum over the sorted frame
        pl.col("daily_strikes").cumsum().alias("strikes_cumulative")
    )
    .filter(pl.col("date") <= date(2013, 12, 31))
)
running_total_strikes

In [ ]:
#question2
damage_state = (
    strikes.groupby("Origin State")
    .agg(pl.col("Cost: Total $").fill_null(0).sum().alias("damage"))
    .with_columns(
        # Global ranking (dense) from highest damage to lowest (window over the whole frame)
        pl.col("damage").rank(method="dense", descending=True).alias("ranking")
    )
    .filter(pl.col("ranking") == 3)
)
damage_state

In [ ]:
#question3
type_species = (
    strikes.groupby(["Aircraft: Type", "Wildlife: Species Group"])
    .agg(pl.col("Cost: Total $").fill_null(0).sum().alias("damage"))
    .with_columns(
        # Rank within each aircraft type
        pl.col("damage")
          .rank(method="dense", descending=True)
          .over("Aircraft: Type")
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 2)
    .sort(["Aircraft: Type", "damage"], descending=[False, True])
)
type_species

In [ ]:
#question4
greatest_strike_increase = (
    strikes.with_columns(
        pl.col("Collision Date and Time").str.to_datetime(strict=False).dt.date().alias("date")
    )
    .groupby("date", maintain_order=True)
    .agg(pl.col("Number of Strikes").fill_null(0).sum().alias("daily_strikes"))
    .sort("date")
    .with_columns(
        pl.col("daily_strikes").shift(1).alias("previous_day")
    )
    .with_columns(
        (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
    )
    .sort("delta_strikes", descending=True)
)
greatest_strike_increase

In [ ]:
#question5
greatest_strike_increase_type = (
    strikes.with_columns(
        pl.col("Collision Date and Time").str.to_datetime(strict=False).dt.date().alias("date")
    )
    .groupby(["Aircraft: Type", "date"], maintain_order=True)
    .agg(pl.col("Number of Strikes").fill_null(0).sum().alias("daily_strikes"))
    .sort(["Aircraft: Type", "date"])
    .with_columns(
        pl.col("daily_strikes").shift(1).over("Aircraft: Type").alias("previous_day")
    )
    .with_columns(
        (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
    )
    .with_columns(
        pl.col("delta_strikes")
          .rank(method="dense", descending=True)
          .over("Aircraft: Type")
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 1)
    .sort("Aircraft: Type")
)
greatest_strike_increase_type